In [1]:
import fitsio
from astropy.io import fits as pyfits

In [2]:
# list of test image files
import glob
files = glob.glob('images/*.fits')
print files

['images/scaled15000x15000.fits', 'images/5000x5000.fits', 'images/1000x1000.fits', 'images/15000x15000.fits', 'images/iris.fits', 'images/10000x10000.fits']


## Timings

Here we choose to test `15000x15000.fits` as the testing image, since it (one of) the largest images we have in this testing set. Differences in speed will be more apparent with larger and larger image sizes. In the tests below, we will testing multiple options that might have an impact of accessing sections of data from image files.

Also, we want caching to speed up the access here, since we might have to extract the data more than once in practice. Hence, ignore the warnings about 'intermediate results being cached' and use the fastest time as the final result.

### PyFits Calls

`PyFits` can be called with a `memmap` option which will tell the loading function whether or not to use a memory-map when loading the image into memory. Using a `memmap` will make access speeds faster; as shown below. [Call API Here](http://pythonhosted.org/pyfits/api/files.html#pyfits.open)

There are no other performance options to this loading function. `PyFits` defaults to using 'readonly' mode, thus it does not need to be specified as so.

In [3]:
# Test PyFits with memmap=True (force PyFits to use a memmap)
# This is default behavior when this keyword is empty --
# i.e. Usually PyFits will automatically try to memmap the image, but
# here we are telling it to use one anyways
with pyfits.open(files[3], memmap=True) as hdul:
    %timeit hdul[0].data[5000:-5000,5000:-5000]

The slowest run took 97.30 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.12 µs per loop


In [4]:
# Test PyFits with memmap=False (force PyFits to NOT use a memmap)
# Loads the file section into memory
with pyfits.open(files[3], memmap=False) as hdul: 
    %timeit hdul[0].data[5000:-5000,5000:-5000]

The slowest run took 642213.43 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 5.01 µs per loop


#### Sections

`pyfits`, as an alternative to memory-mapping, provides [`sections`](http://docs.astropy.org/en/stable/io/fits/usage/image.html#data-sections) for accessing parts of large data files. This is the only alterative for loading files with `bzero` & `bscale` arguments and for those that cannot be loaded (in entirety) into memory. In large parts, this method is superseded by `memmaps` but it is useful to gauge how access speeds vary with this approach.

##### Memmap, no-Memmap, and sections on a image file with bzero/bscale parameters

In [22]:
# example of case where scaled images cannot be memory-mapped
# error is intended to be shown!
try:
    hdul = pyfits.open(files[0], memmap=True)
    %timeit hdul[0].data[5000:-5000,5000:-5000]
except ValueError as ve:
   print ve

Cannot load a memory-mapped image: BZERO/BSCALE/BLANK header keywords present. Set memmap=False.


In [23]:
# try again, but with memmap=False
try:
    hdul = pyfits.open(files[0], memmap=False)
    %timeit hdul[0].data[5000:-5000,5000:-5000]
except ValueError as ve:
   print ve

The slowest run took 755135.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 4.05 µs per loop


In [24]:
# try again, but with memmap=False
# and use sections
try:
    hdul = pyfits.open(files[0], memmap=False)
    %timeit hdul[0].section[5000:-5000,5000:-5000]
except ValueError as ve:
   print ve

1 loop, best of 3: 434 ms per loop


##### Use sections naively on simple image file

In [5]:
# Test PyFits with memmap=True (force PyFits to use a memmap)
with pyfits.open(files[3], memmap=True) as hdul: 
    %timeit hdul[0].section[5000:-5000,5000:-5000]

1 loop, best of 3: 174 ms per loop


In [6]:
# Test PyFits with memmap=False (force PyFits to NOT use a memmap)
with pyfits.open(files[3], memmap=False) as hdul: 
    %timeit hdul[0].section[5000:-5000,5000:-5000]

1 loop, best of 3: 275 ms per loop


### Fitsio Calls

`fitsio` only has the simple loading call with no extra performance specific parameters. It has a `iter_row_buffer` parameter for speeding up loading table row/colummns, but is not intended for reading images.

[Examples on how to call given here, under the 'documentation' (repo description)](https://github.com/esheldon/fitsio) 

[Link to object call in the code](https://github.com/esheldon/fitsio/blob/master/fitsio/fitslib.py#L262)

Just like `pyfits`, `fitsio` defaults to 'readonly' for files.

In [7]:
# Use fitsio with default parameters
# fitsio doesn't have (any!) performance improving options 
with fitsio.FITS(files[3]) as f:
    %timeit f[0][5000:-5000,5000:-5000]
    f.close()

1 loop, best of 3: 237 ms per loop


## Conclusions

`pyfits` has much faster access speeds than `fitsio`. The difference is abyssal, with `pyfits` being $\gt5x10^4$ faster than `fitsio`. This drastic time difference is very strange; implying `fitsio` does not use memory-maps by default (since the timings for `pyfits.sections` is similar in magnitude to those of `fitsio`). Also, should be noted that using `pyfits.sections` is not as fast as normal accessing, and ought to avoided whenever possible. It is the minimum performance benchmark, and only should be used when all other loading methods are inadequate.

As far as I can find, these are the quickest ways to load and access part of an image. If you know of another way to load that may help performance-wise, let me know/add it here.  